In [1]:
import pandas as pd
import numpy as np
import copy
from scipy import stats
from sklearn.metrics.pairwise import cosine_similarity
import operator
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import roc_curve, auc

/Users/rajeevkumar/anaconda/lib/python3.4/site-packages/pandas/computation/__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


In [2]:
# output in a table containing player name, its traded age, predicted war, true war
final_output_hitters = pd.DataFrame()
final_output_pitchers = pd.DataFrame()

In [3]:
hitter_data = pd.read_csv('war_daily_bat.csv', sep=",")
pitcher_data = pd.read_csv('war_daily_pitch.csv', sep=",")
lahman_master = pd.read_csv('../baseballdatabank-master/core/Master.csv')

In [4]:
hitter_data.columns

Index(['name_common', 'age', 'mlb_ID', 'player_ID', 'year_ID', 'team_ID',
       'stint_ID', 'lg_ID', 'PA', 'G', 'Inn', 'runs_bat', 'runs_br', 'runs_dp',
       'runs_field', 'runs_infield', 'runs_outfield', 'runs_catcher',
       'runs_good_plays', 'runs_defense', 'runs_position', 'runs_position_p',
       'runs_replacement', 'runs_above_rep', 'runs_above_avg',
       'runs_above_avg_off', 'runs_above_avg_def', 'WAA', 'WAA_off', 'WAA_def',
       'WAR', 'WAR_def', 'WAR_off', 'WAR_rep', 'salary', 'pitcher', 'teamRpG',
       'oppRpG', 'oppRpPA_rep', 'oppRpG_rep', 'pyth_exponent',
       'pyth_exponent_rep', 'waa_win_perc', 'waa_win_perc_off',
       'waa_win_perc_def', 'waa_win_perc_rep', 'OPS_plus', 'TOB_lg', 'TB_lg'],
      dtype='object')

In [5]:
pitcher_data.columns

Index(['name_common', 'age', 'mlb_ID', 'player_ID', 'year_ID', 'team_ID',
       'stint_ID', 'lg_ID', 'G', 'GS', 'IPouts', 'IPouts_start',
       'IPouts_relief', 'RA', 'xRA', 'xRA_sprp_adj', 'xRA_def_pitcher', 'PPF',
       'PPF_custom', 'xRA_final', 'BIP', 'BIP_perc', 'RS_def_total',
       'runs_above_avg', 'runs_above_avg_adj', 'runs_above_rep',
       'RpO_replacement', 'GR_leverage_index_avg', 'WAR', 'salary', 'teamRpG',
       'oppRpG', 'pyth_exponent', 'waa_win_perc', 'WAA', 'WAA_adj',
       'oppRpG_rep', 'pyth_exponent_rep', 'waa_win_perc_rep', 'WAR_rep',
       'ERA_plus', 'ER_lg'],
      dtype='object')

In [6]:
# ages range [15,50], some are [52 to 58]
hitter_unique_ages = hitter_data['age'].unique()

In [7]:
table_hitter = pd.DataFrame()
table_pitcher = pd.DataFrame()
features_selected = ['name_common', 'mlb_ID', 'age', 'player_ID', 'year_ID', 'team_ID', 'WAR']

In [8]:
table_hitter = copy.deepcopy(hitter_data[features_selected])
table_pitcher = copy.deepcopy(pitcher_data[features_selected])
table_hitter = table_hitter.sort(['age'], ascending=[1])
table_hitter = table_hitter.dropna()
print(len(table_hitter))
table_pitcher = table_pitcher.sort(['age'], ascending=[1])
table_pithcer = table_pitcher.dropna()
print(len(table_pitcher))

102158
44950


In [39]:
current_age = [32]
future_trade_age = [35]

In [10]:
# find all players having data while at age = 30
# hitters_with_traded_age = table_hitter[table_hitter['age'] == future_trade_age]
# hitters_with_traded_age = np.array(hitters_with_traded_age['name_common'])
# hitters_with_traded_age

array(['Paul Splittorff', 'Tim Leary', 'Steve Lake', ..., 'Ellis Burks',
       'Alvaro Espinoza', 'Billy Shindle'], dtype=object)

In [11]:
list_of_player_current_age = np.array(table_hitter[table_hitter['age'] == current_age]['mlb_ID'])
len(list_of_player_current_age)

4771

In [50]:
def calculate_traded_vec(traded_player, current_age):
    # player [hitter] whose future trade value is to be calculated
    traded_player_vec = []
    ages = sorted(np.array(traded_player['age']))
    for age in range(16, int(current_age)+1):
        age = 1.0*age
        val = np.array(traded_player[traded_player['age'] == age]['WAR'])
        if len(val) > 0:
            traded_player_vec.append(max(val))
        else:
            traded_player_vec.append(0.01)
    mean_val = np.mean(traded_player_vec)
    var_val = np.std(traded_player_vec)
    traded_player_vec.append(mean_val)
    traded_player_vec.append(var_val)
    return traded_player_vec
    

In [151]:
def calculated_weighted_WAR(top_similar_players, future_trade_age):
    # cosine sim weighted future WAR value
    predicted_WAR = 0.0
    total_sim = 0.0
    for similar_player in top_similar_players:
        sim_score = similar_player[1][0][0]
        if sim_score > 0.75:
            player = similar_player[0]
            player_data = table_hitter[table_hitter['name_common'] == player]
            player_WAR_at_future_year = np.array(player_data[player_data['age'] == future_trade_age]['WAR'])
            if len(player_WAR_at_future_year) > 0: 
                player_WAR_at_future_year = max(player_WAR_at_future_year)
                total_sim += sim_score
                predicted_WAR += sim_score*player_WAR_at_future_year
    #         print(player, player_WAR_at_future_year)
    if total_sim == 0:
        final_predicted_WAR = 0.01
    else:
        final_predicted_WAR = predicted_WAR/total_sim
    print('Future predicted war is in range [ %f , %f ] in age %d for player %s' %(final_predicted_WAR -0.5, final_predicted_WAR + 0.5, future_trade_age, traded_player_name))
    return final_predicted_WAR

    
    
    

In [52]:
def calculated_similar_player_vec(traded_player_vec,traded_player_name, current_age, future_trade_age):
    list_player_vec = []
    player_dic_sim = {}
    for player in hitters_with_traded_age:
        if player != traded_player_name:
            player_vec = []
            player_data = table_hitter[table_hitter['name_common'] == player]
            for age in range(16, int(current_age)+1):
                age = 1.0*age
                val = np.array(player_data[player_data['age'] == age]['WAR'])
                if len(val) > 0:
                    player_vec.append(max(val))
                else:
                    player_vec.append(0.01)
            mean_val = np.mean(player_vec)
            var_val = np.std(player_vec)
            player_vec.append(mean_val)
            player_vec.append(var_val)
            list_player_vec.append(player_vec)
            cos_sim = cosine_similarity(traded_player_vec, player_vec)
            player_dic_sim[player] = cos_sim
    top_similar_players = sorted(player_dic_sim.items(), key=operator.itemgetter(1), reverse=True)
    output = calculated_weighted_WAR(top_similar_players, future_trade_age)
    return output


In [152]:
count = 100
player_names = []
player_actual_WAR = []
player_predicted_WAR = []
for i in range(62, count):
#     mlb_ID = list_of_player_current_age[i]
    mlb_ID = traded_MLB_IDS[i]
#     mlb_ID = 77582.0
    current_age_v = current_age[i]
    print(current_age_v)
    future_trade_age_v = future_trade_age[i]
    print(future_trade_age_v)
    traded_player = table_hitter[table_hitter['mlb_ID'] == mlb_ID]
    traded_player_name = np.array(traded_player['name_common'])
#     print(traded_player_name)
    traded_player_name = np.unique(traded_player_name)[0]
#     print(traded_player_name)
    
    traded_player_vec = calculate_traded_vec(traded_player, current_age_v)
    predicted_WAR = calculated_similar_player_vec(traded_player_vec,traded_player_name, current_age_v, future_trade_age_v)
    actual_WAR = np.array(traded_player[traded_player['age'] == future_trade_age_v]['WAR'])
    if len(actual_WAR) > 0:
        actual_WAR = max(actual_WAR)
    else:
        actual_WAR = 0.01
    player_names.append(traded_player_name)
    player_actual_WAR.append(actual_WAR)
    player_predicted_WAR.append(predicted_WAR)
final_output_hitters = pd.DataFrame({'Hitters': player_names, 'Actual WAR': player_actual_WAR, 'Predicted WAR': player_predicted_WAR})


33.0
35.0
Future predicted war is in range [ -0.490000 , 0.510000 ] in age 35 for player Cub Stricker
33.0
35.0
Future predicted war is in range [ -0.490000 , 0.510000 ] in age 35 for player Cub Stricker
32.0
34.0
Future predicted war is in range [ 0.840105 , 1.840105 ] in age 34 for player Yank Robinson
27.0
29.0
Future predicted war is in range [ 0.043225 , 1.043225 ] in age 29 for player Adonis Terry
20.0
22.0
Future predicted war is in range [ 1.521410 , 2.521410 ] in age 22 for player Joe Kelley
20.0
22.0
Future predicted war is in range [ 1.521410 , 2.521410 ] in age 22 for player Joe Kelley
26.0
28.0
Future predicted war is in range [ 2.633326 , 3.633326 ] in age 28 for player George Van Haltren
26.0
28.0
Future predicted war is in range [ 2.633326 , 3.633326 ] in age 28 for player George Van Haltren
33.0
35.0
Future predicted war is in range [ 0.665345 , 1.665345 ] in age 35 for player Cliff Carroll
30.0
32.0
Future predicted war is in range [ 0.022714 , 1.022714 ] in age 32 fo

In [192]:
len(player_names)

38

In [193]:
len(player_actual_WAR)

38

In [194]:
len(player_predicted_WAR)

38

In [195]:
final_output_hitters = pd.DataFrame({'Hitters': player_names, 'Actual WAR': player_actual_WAR, 'Predicted WAR': player_predicted_WAR})

In [196]:
final_output_hitters

,Actual WAR,Hitters,Predicted WAR
0,0.01,Cub Stricker,0.010000
1,0.01,Cub Stricker,0.010000
2,0.01,Yank Robinson,1.340105
3,0.45,Adonis Terry,0.543225
4,6.52,Joe Kelley,2.021410
5,6.52,Joe Kelley,2.021410
6,1.82,George Van Haltren,3.133326
7,1.82,George Van Haltren,3.133326
8,0.01,Cliff Carroll,1.165345
9,1.43,Joe Quinn,0.522714


In [185]:
final_output_hitters = final_output_hitters.drop_duplicates()
final_output_hitters

,Actual WAR,Hitters,Predicted WAR
0,0.01,Cub Stricker,0.010000
2,0.01,Yank Robinson,1.340105
3,0.45,Adonis Terry,0.543225
4,6.52,Joe Kelley,2.021410
6,1.82,George Van Haltren,3.133326
7,1.82,George Van Haltren,3.133326
8,0.01,Cliff Carroll,1.165345
9,1.43,Joe Quinn,0.522714
10,0.01,Danny Richardson,2.570192
11,0.01,Kid Summers,0.785193


In [197]:
final_output_hitters['Current_Age'] = current_age[62:100]

In [191]:
len(final_output_hitters)

34

In [198]:
final_output_hitters['Future_Age'] = future_trade_age[62:100]

In [199]:
final_output_hitters['MLB_ID'] = traded_MLB_IDS[62:100]

In [200]:
final_output_hitters['Transaction ID'] =  transaction_IDS[62:100]

In [207]:
final_output_hitters = final_output_hitters.drop_duplicates(keep=False)
final_output_hitters = final_output_hitters.drop_duplicates(keep=False)
final_output_hitters

,Actual WAR,Hitters,Predicted WAR,Current_Age,Future_Age,MLB_ID,Transaction ID
0,0.01,Cub Stricker,0.010000,33.0,35.0,122871.0,48231.0
2,0.01,Yank Robinson,1.340105,32.0,34.0,121325.0,48231.0
3,0.45,Adonis Terry,0.543225,27.0,29.0,123202.0,48231.0
4,6.52,Joe Kelley,2.021410,20.0,22.0,116926.0,48232.0
6,1.82,George Van Haltren,3.133326,26.0,28.0,123646.0,48232.0
7,1.82,George Van Haltren,3.133326,26.0,28.0,123646.0,48232.0
8,0.01,Cliff Carroll,1.165345,33.0,35.0,112051.0,48233.0
9,1.43,Joe Quinn,0.522714,30.0,32.0,120845.0,48233.0
10,0.01,Danny Richardson,2.570192,30.0,32.0,121155.0,48234.0
11,0.01,Kid Summers,0.785193,25.0,27.0,122979.0,48235.0


In [208]:
final_output_hitters.to_csv('Predicted_WAR_38_Hitters_1.csv')

### Results :-
##### Future predicted war is 2.107890 in age 32 for player Brady Clark and actual is 2.3
##### Future predicted war is 1.470445 in age 32 for player Mike Benjamin and actual is 2.07
Future predicted war is in range 1.34 in age 35 for player Ivan Rodriguez and actual is 1.36
Future predicted war is in range 1.7 in age 35 for player Mark McGwire and actual is 5.2

In [144]:
from sklearn.metrics import mean_squared_error
from math import sqrt

# new_final_output_hitters = final_output_hitters[final_output_hitters['Actual WAR'] != 0.01]
new_final_output_hitters = copy.deepcopy(final_output_hitters)
y_actual = np.array(new_final_output_hitters['Actual WAR'])
y_predicted = np.array(new_final_output_hitters['Predicted WAR'])
new_final_output_hitters['low_interval'] = new_final_output_hitters['Predicted WAR'] - 0.5
new_final_output_hitters['high_interval'] = new_final_output_hitters['Predicted WAR'] + 0.5
# y_predicted = np.array(new_final_output_hitters['low_interval'])
# y_predicted = np.array(new_final_output_hitters['high_interval'])
rms = sqrt(mean_squared_error(y_actual, y_predicted))
print('Root mean squared error is : ', rms)

Root mean squared error is :  1.6012861245929257


/Users/rajeevkumar/anaconda/lib/python3.4/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [176]:
new_final_output_hitters[new_final_output_hitters['Actual WAR'] != 0.01]

,Actual WAR,Hitters,Predicted WAR,low_interval,high_interval
2,-0.04,Mike Morgan,-0.085479,-0.585479,0.414521
3,0.04,Brett Tomko,0.338539,-0.161461,0.838539
5,0.29,Frank Catalanotto,1.345845,0.845845,1.845845
7,0.04,Ryan Franklin,1.385055,0.885055,1.885055
11,-0.31,Gene Baker,0.925335,0.425335,1.425335
17,-0.04,Willie Jones,1.528119,1.028119,2.028119
18,0.44,Harry Brecheen,0.208005,-0.291995,0.708005
20,0.21,Dave Parker,1.125239,0.625239,1.625239
29,8.97,Willie Mays,1.398361,0.898361,1.898361
31,-0.46,Paul Bako,-0.076725,-0.576725,0.423275


In [182]:
table_hitter[table_hitter['name_common'] == 'Willie Mays']

,name_common,mlb_ID,age,player_ID,year_ID,team_ID,WAR
58771,Willie Mays,118495.0,20.0,mayswi01,1951,NYG,3.94
58772,Willie Mays,118495.0,21.0,mayswi01,1952,NYG,1.25
58773,Willie Mays,118495.0,23.0,mayswi01,1954,NYG,10.61
58774,Willie Mays,118495.0,24.0,mayswi01,1955,NYG,9.03
58775,Willie Mays,118495.0,25.0,mayswi01,1956,NYG,7.58
58776,Willie Mays,118495.0,26.0,mayswi01,1957,NYG,8.29
58777,Willie Mays,118495.0,27.0,mayswi01,1958,SFG,10.24
58778,Willie Mays,118495.0,28.0,mayswi01,1959,SFG,7.79
58779,Willie Mays,118495.0,29.0,mayswi01,1960,SFG,9.48
58780,Willie Mays,118495.0,30.0,mayswi01,1961,SFG,8.72


In [200]:
player = 'Matt Williams'
mlb_ID = [121358.0, 118743.0, 110183.0]

In [201]:
table_hitter[table_hitter['name_common'] == player]

,name_common,mlb_ID,age,player_ID,year_ID,team_ID,WAR
1314,Roberto Alomar,110183.0,20.0,alomaro01,1988,SDP,4.33
1315,Roberto Alomar,110183.0,21.0,alomaro01,1989,SDP,4.43
1316,Roberto Alomar,110183.0,22.0,alomaro01,1990,SDP,3.38
1317,Roberto Alomar,110183.0,23.0,alomaro01,1991,TOR,4.54
1318,Roberto Alomar,110183.0,24.0,alomaro01,1992,TOR,6.64
1319,Roberto Alomar,110183.0,25.0,alomaro01,1993,TOR,6.10
1320,Roberto Alomar,110183.0,26.0,alomaro01,1994,TOR,2.24
1321,Roberto Alomar,110183.0,27.0,alomaro01,1995,TOR,2.68
1322,Roberto Alomar,110183.0,28.0,alomaro01,1996,BAL,5.27
1323,Roberto Alomar,110183.0,29.0,alomaro01,1997,BAL,3.43


In [92]:
columns = ['playerID', 'retroID', 'bbrefID', 'nameGiven']
lahman_dataset = lahman_master[columns]
lahman_dataset.columns

Index(['playerID', 'retroID', 'bbrefID', 'nameGiven'], dtype='object')

In [93]:
trade_data = pd.read_csv('../tranDB/tran.txt', sep=",", header=None)
trade_data.columns = ['primary-date', 'time', 'approximate-indicator', 'secondary-date', 'approximate-indicator', 'transaction-ID',
'player', 'type', 'from-team', 'from-league', 'to-team', 'to-league', 'draft-type', 'draft-round', 'pick-number', 'info']
trade_data = trade_data[trade_data['type'] == 'T ']

In [94]:
def find_year(z):
    year = z[0:4]
    return int(year)
new_trade_data = pd.DataFrame()
new_trade_data['year_ID'] = trade_data['primary-date'].apply(lambda x : find_year(str(x)))
new_trade_data['transaction-ID'] = trade_data['transaction-ID']
new_trade_data['player'] = trade_data['player']

In [95]:
def map_playerID(x):
    if x in lahman_dic:
        return lahman_dic[x]
    else:
        return x
    
lahman_dic = lahman_dataset.set_index('retroID')['nameGiven'].to_dict()
lahman_name_dic = lahman_master.set_index('nameGiven')['bbrefID'].to_dict()
new_trade_data['player'] = new_trade_data['player'].apply(lambda x: map_playerID(x))
    

In [104]:
def map_playerID(x):
    if x in lahman_name_dic:
        return lahman_name_dic[x]
    else:
        return x
lahman_name_dic = lahman_master.set_index('bbrefID')['nameGiven'].to_dict()
table_hitter['player Name'] = table_hitter['player_ID'].apply(lambda x: map_playerID(x))

In [105]:
table_hitter['key'] = table_hitter['player Name']+'_'+table_hitter['year_ID'].apply(lambda x : str(x))

In [106]:
new_trade_data['key'] = new_trade_data['player'] +'_'+ new_trade_data['year_ID'].apply(lambda x: str(x))

In [107]:
possible_trades = {}
for row in new_trade_data.iterrows():
    possible_trades[row[1][3]] = row[1][1]


In [108]:
sample = table_hitter.head(5)

In [109]:
bool_trade = []
mlb_transactionID = []
for row in table_hitter.iterrows():
    if row[1][8] in possible_trades:
        bool_trade.append(1)
        mlb_transactionID.append(possible_trades[row[1][8]])
    else:
        bool_trade.append(0)
        mlb_transactionID.append(None)

In [110]:
table_hitter['Traded or not'] = bool_trade
table_hitter['MLB Transaction ID'] = mlb_transactionID

In [111]:
table_hitter

,name_common,mlb_ID,age,player_ID,year_ID,team_ID,WAR,player Name,key,Traded or not,MLB Transaction ID
67897,Joe Nuxhall,119872.0,15.0,nuxhajo01,1944,CIN,0.00,Joseph Henry,Joseph Henry_1944,0,NaN
52556,Leonidas Lee,117605.0,16.0,leele01,1877,STL,0.05,Leonidas Pyrrhus,Leonidas Pyrrhus_1877,0,NaN
22763,Jim Derrington,113286.0,16.0,derriji01,1956,CHW,0.04,Charles James,Charles James_1956,0,NaN
7600,Coonie Blank,111075.0,16.0,blankco01,1909,STL,-0.04,Frank Ignatz,Frank Ignatz_1909,0,NaN
12902,Putsy Caballero,111846.0,16.0,cabalpu01,1944,PHI,-0.20,Ralph Joseph,Ralph Joseph_1944,0,NaN
82230,Carl Scheib,121788.0,16.0,scheica01,1943,PHA,-0.09,Carl Alvin,Carl Alvin_1943,0,NaN
10352,Jim Britt,111484.0,16.0,brittji01,1872,BRA,-0.23,James Edward,James Edward_1872,0,NaN
71159,Frank Pearce,120328.0,16.0,pearcfr01,1876,LOU,-0.04,Franklin Johnson,Franklin Johnson_1876,0,NaN
11144,Tommy Brown,111594.0,16.0,brownto02,1944,BRO,-2.05,Thomas Michael,Thomas Michael_1944,0,NaN
60426,Roger McKee,118770.0,16.0,mckeero01,1943,PHI,0.03,Roger Hornsby,Roger Hornsby_1943,0,NaN


In [89]:
table_Traded_hitters = table_hitter[table_hitter['Traded or not'] == 1]

In [116]:
table_Traded_hitters = table_Traded_hitters.sort(['MLB Transaction ID'], ascending=[1])

In [117]:
table_Traded_hitters

,name_common,mlb_ID,age,player_ID,year_ID,team_ID,WAR,player Name,key,Traded or not,MLB Transaction ID
21996,Pat Deasley,113176.0,27.0,deaslpa01,1885,NYG,1.13,Thomas H.,Thomas H._1885,1,48185.0
18174,John Connor,112587.0,23.0,connojo02,1885,LOU,-0.10,John,John_1885,1,48185.0
18173,John Connor,112587.0,23.0,connojo02,1885,BUF,-0.06,John,John_1885,1,48185.0
75538,Toad Ramsey,120919.0,20.0,ramseto01,1885,LOU,-0.26,Thomas H.,Thomas H._1885,1,48185.0
62667,Jocko Milligan,119136.0,23.0,millijo01,1885,PHA,2.19,John,John_1885,1,48185.0
28581,John Fischer,114158.0,28.0,fishe01,1885,BUF,-0.06,John,John_1885,1,48185.0
68041,Tom O'Brien,119908.0,25.0,o'brito01,1885,BAL,0.10,Thomas H.,Thomas H._1885,1,48185.0
18311,Dick Conway,112612.0,20.0,conwadi01,1886,BAL,0.03,Richard Butler,Richard Butler_1886,1,48195.0
18309,Bill Conway,112614.0,24.0,conwabi01,1886,BAL,-0.03,William F.,William F._1886,1,48195.0
68105,Pat O'Connell,119920.0,25.0,o'conpa01,1886,BAL,-0.99,Patrick H.,Patrick H._1886,1,48195.0


In [118]:
traded_MLB_IDS = np.array(table_Traded_hitters['mlb_ID'])

In [138]:
transaction_IDS = np.array(table_Traded_hitters['MLB Transaction ID'])

In [150]:
transaction_IDS[62:]

array([ 48231.,  48231.,  48231., ...,  77616.,  77618.,  77618.])

In [119]:
traded_MLB_IDS

array([ 113176.,  112587.,  112587., ...,  607359.,  521230.,  445926.])

In [120]:
current_age = np.array(table_Traded_hitters['age'])
future_trade_age = np.array(table_Traded_hitters['age'] + 2)

In [121]:
current_age

array([ 27.,  23.,  23., ...,  27.,  26.,  31.])

In [122]:
future_trade_age

array([ 29.,  25.,  25., ...,  29.,  28.,  33.])